In [ ]:
! pip install torch tensorflow transformers

In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from transformers import BertForSequenceClassification, BertTokenizer
import torch

2024-01-08 15:04:24.195415: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 15:04:24.195459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 15:04:24.196898: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 15:04:24.204339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 15:04:25.069684: W tensorflow/compiler/tf2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
with tf.device('/cpu:0'):
    model = TFAutoModel.from_pretrained("vinai/phobert-base-v2", from_pt=True)
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

2024-01-08 15:06:24.298256: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 196611072 exceeds 10% of free system memory.
2024-01-08 15:06:24.454889: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 196611072 exceeds 10% of free system memory.
2024-01-08 15:06:24.509614: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 196611072 exceeds 10% of free system memory.
2024-01-08 15:06:25.919503: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 196611072 exceeds 10% of free system memory.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TF

In [5]:
class BERTForClassification(tf.keras.Model):
    
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.bert.trainable = False
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [8]:
with tf.device('/cpu:0'):
    classifier = BERTForClassification(model, num_classes=2)
    classifier.load_weights('./model/model1/model_weights')

2024-01-08 15:09:05.312768: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 196611072 exceeds 10% of free system memory.


In [12]:
text = "context: Thủ tục hành chính tên Cấp lại giấy chứng sinh đối với trường hợp bị mất hoặc hư hỏng với mã 1.002150.000.00.00.H13 được thực hiện theo trình tự thực hiện như sau \nquestion: đăng ký giấy cấp lại căn cước công dân cần hồ sơ gì?"
tokenized_text = tokenizer([text], return_tensors="tf", padding=True, truncation=True, max_length=512)

In [13]:
with tf.device('/cpu:0'):
    output = classifier.call(tokenized_text)
output

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.26601812, 0.7339819 ]], dtype=float32)>